In [3]:
import warnings
warnings.filterwarnings("ignore")

# Alexnet

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

data_dir = "music_genre_dataset/Data/images_original/"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize images for AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalization
])

dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
labels = [label for _, label in dataset]

train_data, test_data, train_label, test_label = train_test_split(dataset, labels, test_size=0.2, random_state=42, stratify=labels)


# Create data loaders for training and testing
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [10]:
model = torchvision.models.alexnet(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
model.classifier[6] = torch.nn.Linear(4096, 10)  # 10 classes


In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cuda')

In [16]:
num_epochs = 10 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(data_loader)}')

print('Training complete')

Epoch 1/10, Loss: 1.8501302786171436
Epoch 2/10, Loss: 1.522880993783474
Epoch 3/10, Loss: 1.4377997256815434
Epoch 4/10, Loss: 1.3866420239210129
Epoch 5/10, Loss: 1.3119575940072536
Epoch 6/10, Loss: 1.3354383558034897
Epoch 7/10, Loss: 1.2596392650157213
Epoch 8/10, Loss: 1.2419206257909536
Epoch 9/10, Loss: 1.234517516568303
Epoch 10/10, Loss: 1.2786836959421635
Training complete


In [17]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')


Accuracy on the test set: 65.0%


# VGG16

In [23]:
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = torch.nn.Linear(4096, 10)

In [24]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

print('Training complete')

Epoch 1/10, Loss: 2.3518637084960936
Epoch 2/10, Loss: 2.341915683746338
Epoch 3/10, Loss: 2.330120315551758
Epoch 4/10, Loss: 2.3215833854675294
Epoch 5/10, Loss: 2.3442965507507325
Epoch 6/10, Loss: 2.3587585067749024
Epoch 7/10, Loss: 2.350939865112305
Epoch 8/10, Loss: 2.3424289512634275
Epoch 9/10, Loss: 2.3444355964660644
Epoch 10/10, Loss: 2.336500873565674
Training complete


In [25]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set with VGG16: {100 * correct / total}%')

Accuracy on the test set with VGG16: 18.5%


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Define the path to your dataset
data_dir = 'path/to/images_original/'

# Define data transformations for data augmentation (if needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create a dataset
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training and testing sets (80% train, 20% test)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Load the pre-trained ResNet-50 model
model = torchvision.models.resnet50(pretrained=True)

# Modify the final classification layer for your specific case (10 classes)
model.fc = torch.nn.Linear(2048, 10)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

print('Training complete')

# Evaluate the model on the testing set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy}%')
